In [0]:
%run "./init"

In [0]:
import mlflow
import pandas as pd

client = mlflow.tracking.MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

if experiment is None:
    display(f"Experiment '{experiment_name}' does not exist. Please run Notebook 1.")
else:
    runs = client.search_runs(experiment.experiment_id)

    # Extract relevant metrics
    df = pd.DataFrame([{
        "run_id": r.info.run_id,
        "scenario": r.data.params.get("scenario"),
        "factory_down": r.data.params.get("factory_down"),
        "cost": r.data.metrics.get("total_cost"),
        "unmet": r.data.metrics.get("total_unmet_demand")
    } for r in runs])
    df = df.sort_values("cost")
    display(df)

In [0]:
baseline_cost = df[df["scenario"] == "Baseline"]["cost"].values[0]
df["regret"] = df["cost"] - baseline_cost
df.sort_values("regret", ascending=False)

In [0]:
import matplotlib.pyplot as plt

plt.bar(df["scenario"], df["regret"])
plt.ylabel("Regret (vs baseline)")
plt.title("Worst-case cost increase per disruption scenario")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()

In [0]:
def find_tts(factory_down, max_days=10):
    for t in range(1, max_days + 1):
        cap = factory_capacity.copy()
        extra = extra_capacity.copy()
        cap[factory_down] = 0
        extra[factory_down] = 0
        
        model = build_model(cap, extra)
        solver = pyo.SolverFactory("cbc")
        solver.solve(model)
        unmet = sum(model.unmet[c].value for c in model.C)
        
        if unmet > 0:
            return t - 1  # Last survivable period
    return max_days

tts_results = {f: find_tts(f) for f in factories}
tts_results


In [0]:
# We'll simulate pre-positioning 20 units of safety stock at F1 and F2

def run_mitigation(factory_down, safety_stock):
    cap = factory_capacity.copy()
    extra = extra_capacity.copy()
    
    # Apply disruption
    cap[factory_down] = 0
    extra[factory_down] = 0
    
    # Add safety stock as extra capacity at the other factory
    for f in factories:
        if f != factory_down:
            extra[f] += safety_stock
    
    model = build_model(cap, extra)
    solver = pyo.SolverFactory("cbc")
    solver.solve(model)
    
    cost = pyo.value(model.obj)
    unmet = sum(model.unmet[c].value for c in model.C)
    return {"factory_down": factory_down, "unmet": unmet, "cost": cost}

# Test mitigation for both factories
mitigation_f1 = run_mitigation("F1", safety_stock=20)
mitigation_f2 = run_mitigation("F2", safety_stock=20)

mitigation_f1, mitigation_f2

In [0]:
report = pd.DataFrame([
    {"factory": "F1", "regret": df[df["scenario"]=="F1_Down"]["regret"].values[0], "tts": tts_results["F1"], "unmet_post_mitigation": mitigation_f1["unmet"]},
    {"factory": "F2", "regret": df[df["scenario"]=="F2_Down"]["regret"].values[0], "tts": tts_results["F2"], "unmet_post_mitigation": mitigation_f2["unmet"]},
])
report
